In [1]:
import pyarrow

In [2]:
# Import the libraries we will need
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

import findspark
findspark.init()

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
import pyspark.pandas as ps

# create the Spark Session
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

# # create the Spark Context
# sc = spark.sparkContext

c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# friend = spark.read.csv("hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt", sep='\t')
# friend = friend.select(col("_c0").alias("User"), col("_c1").alias("Friends"))
# friend = friend.withColumn('Friends', split('Friends', ','))
# friend.show(5)

In [7]:
friend = spark.read.csv("hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt", sep='\t')
friend = friend.pandas_api()
friend = friend.rename({'_c0': 'User', '_c1': 'Friends'}, axis = 1)
friend['User'] = friend['User'].astype(int)
friend['Friends'] = friend['Friends'].apply(lambda x: set([]) if (x is None) else set(x.split(','))) # split the index of friends into list

# friend.head()

no_friend = friend[friend['Friends'].apply(len) == 0].rename({'Friends': 'Recommendations'}, axis = 1) # filter users who have no friend
no_friend['Recommendations'] = no_friend['Recommendations'].apply(lambda x: list(x))

friend = friend[friend['Friends'].apply(len) > 0] # filter users who have at least one friend
friend = friend.iloc[:100] # FIX ME: SUBSAMPLING
friend

,User,Friends
0,0,"[18, 14, 17, 34, 50, 8, 22, 37, 46, 38, 72, 77..."
1,1,"[13845, 35648, 49985, 10623, 30691, 32317, 448..."
2,2,"[16659, 6893, 41878, 117, 2755, 0, 24714, 1220..."
3,3,"[12636, 41, 0, 1532, 38737, 12, 27552, 55, 13185]"
4,4,"[18, 14, 42697, 8, 74, 19068, 72, 77, 80, 2459..."
5,5,"[43262, 23527, 0, 41457, 32503, 44846, 31140, ..."
6,6,"[2203, 33080, 9847, 35785, 543, 15294, 575, 25..."
7,7,"[0, 40218, 21843, 31993, 1357, 40433]"
8,8,"[72, 0, 33380, 83, 4, 85, 46, 38, 24777]"
9,9,"[18972, 0, 6085, 19269]"


In [10]:
friend['User'].unique().values

c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\pandas\generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
           9,    10,    11,    12,    13,    14,    15,    16,    17,
          18,    19,    20,    21,    22,    23,    24,    25,    26,
          27,    28,    29,    30,    31,    32,    33,    34,    35,
          36,    37,    38,    39,    40,    41,    42,    43,    44,
          45,    46,   135,  2409,  8715,  8932, 10623, 12347, 12846,
       13840, 13845, 14005, 20075, 21556, 22939, 23520, 28193, 29724,
       29791, 29826, 30691, 31232, 31435, 32317, 32489, 34394, 35589,
       35605, 35606, 35613, 35633, 35648, 35678, 38737, 43447, 44846,
       44887, 49226, 49985,   117,  1220,  2755, 12453, 24539, 24714,
       41456, 45046, 49927,    55,  1532, 12636, 13185, 27552,    72,
          80])

In [12]:
all_user_id = set(friend['User'].unique().to_numpy())
unfriend = friend.copy()
unfriend['Unfriends'] = friend['Friends'].apply(lambda x: all_user_id - set(x))
unfriend = unfriend.drop('Friends', axis=1)
unfriend = unfriend.explode('Unfriends')
unfriend['Unfriends'] = unfriend['Unfriends'].astype(int)
unfriend['Mutual Friends'] = unfriend.apply(lambda row: 
                                            (friend[friend['User'] == row['User']]['Friends'].iloc[0]) & 
                                            (friend[friend['User'] == row['Unfriends']]['Friends'].iloc[0]), axis=1)
unfriend['Num of Mutual Friends'] = unfriend['Mutual Friends'].apply(len)
unfriend.head()

c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


PySparkTypeError: [CANNOT_ACCEPT_OBJECT_IN_TYPE] `IntegerType()` can not accept object `0` in type `int32`.

In [5]:
mutual_friend = unfriend.sort_values(['User', 'Num of Mutual Friends', 'Unfriends'], ascending = False).groupby('User').head(10)
mutual_friend.head(20)

,User,Unfriends,Mutual Friends,Num of Mutual Friends
83,49985,49985,"{19138, 44854, 30313, 42003, 41423, 33072, 385...",55
83,49985,35678,"{35621, 1, 23690, 21556, 35630, 14112, 25228, ...",8
83,49985,35633,"{35621, 1, 21556, 35562, 35630, 34406, 41433, ...",8
83,49985,35613,"{35621, 1, 23690, 21556, 38737, 33072, 35630, ...",8
83,49985,35605,"{35621, 1, 35562, 35630, 25228, 49917}",6
83,49985,23520,"{35621, 35565, 1, 19138, 27536, 41433}",6
83,49985,21556,"{35621, 1, 23690, 49546, 35630, 25228}",6
83,49985,20075,"{35621, 21556, 1, 35630, 25228}",5
83,49985,1,"{27536, 21556, 31232, 38737, 34406}",5
83,49985,49226,"{35621, 27536, 1, 35630}",4


In [6]:
recommendation = mutual_friend.groupby('User')['Unfriends'].agg(lambda x: x.to_list()).reset_index().rename({'Unfriends': 'Recommendations'}, axis = 1)
recommendation

,User,Recommendations
0,0,"[0, 12, 4, 38, 16, 28, 8, 38737, 30, 24]"
1,1,"[1, 20, 35633, 35613, 21556, 35678, 35606, 200..."
2,2,"[2, 5, 1, 49927, 23520, 13845, 8715, 2755, 122..."
3,3,"[3, 41, 12, 0, 38737, 1532, 55, 38, 30, 29]"
4,4,"[4, 0, 15, 14, 80, 72, 46, 38, 30, 27]"
...,...,...
95,44887,"[44887, 31232, 12846, 1, 44846, 34394, 29724, ..."
96,45046,"[45046, 49927, 41456, 24714, 24539, 12453, 275..."
97,49226,"[49226, 35613, 35605, 35678, 35633, 20075, 215..."
98,49927,"[49927, 135, 117, 45046, 41456, 35589, 24714, ..."


In [9]:
ps.concat([final, no_friend], ignore_index=True)

TypeError: cannot concatenate object of type 'list; only ps.Series and ps.DataFrame are valid

In [7]:
final = ps.merge(friend, recommendation, on = 'User')
final = final.drop('Friends', axis = 1)
final = ps.concat([final, no_friend], ignore_index=True).sort_values('User', ascending=True)

TypeError: cannot concatenate object of type 'list; only ps.Series and ps.DataFrame are valid